In [4]:
from workflow.manip import *
import pandas as pd
import duckdb

In [2]:
con = duckdb.connect(database = "database/real_estate.db", read_only = False)
df_data = con.execute("SELECT * FROM immo_france").df()

In [3]:
df_geo = con.execute("SELECT * FROM zone_geographique").df()

In [4]:
df_geo.head()


,code_departement,nom_departement,code_region,nom_region
0,01,Ain,84,Auvergne-Rhône-Alpes
1,02,Aisne,32,Hauts-de-France
2,03,Allier,84,Auvergne-Rhône-Alpes
3,04,Alpes-de-Haute-Provence,93,Provence-Alpes-Côte d'Azur
4,05,Hautes-Alpes,93,Provence-Alpes-Côte d'Azur


In [5]:
df_geo.nom_departement.unique()

array(['Ain', 'Aisne', 'Allier', 'Alpes-de-Haute-Provence',
       'Hautes-Alpes', 'Alpes-Maritimes', 'Ardèche', 'Ardennes', 'Ariège',
       'Aube', 'Aude', 'Aveyron', 'Bouches-du-Rhône', 'Calvados',
       'Cantal', 'Charente', 'Charente-Maritime', 'Cher', 'Corrèze',
       "Côte-d'Or", "Côtes-d'Armor", 'Creuse', 'Dordogne', 'Doubs',
       'Drôme', 'Eure', 'Eure-et-Loir', 'Finistère', 'Corse-du-Sud',
       'Haute-Corse', 'Gard', 'Haute-Garonne', 'Gers', 'Gironde',
       'Hérault', 'Ille-et-Vilaine', 'Indre', 'Indre-et-Loire', 'Isère',
       'Jura', 'Landes', 'Loir-et-Cher', 'Loire', 'Haute-Loire',
       'Loire-Atlantique', 'Loiret', 'Lot', 'Lot-et-Garonne', 'Lozère',
       'Maine-et-Loire', 'Manche', 'Marne', 'Haute-Marne', 'Mayenne',
       'Meurthe-et-Moselle', 'Meuse', 'Morbihan', 'Moselle', 'Nièvre',
       'Nord', 'Oise', 'Orne', 'Pas-de-Calais', 'Puy-de-Dôme',
       'Pyrénées-Atlantiques', 'Hautes-Pyrénées', 'Pyrénées-Orientales',
       'Bas-Rhin', 'Haut-Rhin', 'Rhône',

In [6]:
df_data.head(3)

,no_disposition,date_mutation,nature_mutation,valeur_fonciere,code_voie,voie,code_postal,commune,code_departement,code_commune,nombre_de_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces
0,1,2023-01-05,Vente,1070000.0,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,01,354,1,3.0,Dépendance,0.0,0.0
1,1,2023-01-05,Vente,1070000.0,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,01,354,1,3.0,Dépendance,0.0,0.0
2,1,2023-01-05,Vente,1070000.0,0124,DES HETRES,1630.0,ST-GENIS-POUILLY,01,354,1,2.0,Appartement,233.0,8.0


In [7]:
df_data.commune.unique()

array(['ST-GENIS-POUILLY', 'SERRIERES-SUR-AIN', 'SAINT-JEAN-DE-NIOST',
       ..., 'PARIS 07', 'PARIS 16', 'PARIS 05'], dtype=object)

# Ajout d'une colonne avec jour de la semaine

In [4]:
df_fimo.head()

,date_mutation,jour_semaine,nature_mutation,valeur_fonciere,code_postal,commune,code_departement,code_commune,type_local,surface_reelle_bati,nombre_pieces
0,2023-01-05,4,Vente,1070000.0,1630.0,ST-GENIS-POUILLY,01,354,Dépendance,0.0,0.0
1,2023-01-05,4,Vente,1070000.0,1630.0,ST-GENIS-POUILLY,01,354,Dépendance,0.0,0.0
2,2023-01-05,4,Vente,1070000.0,1630.0,ST-GENIS-POUILLY,01,354,Appartement,233.0,8.0
3,2023-01-03,2,Vente,152200.0,1450.0,SERRIERES-SUR-AIN,01,404,Maison,64.0,3.0
4,2023-01-05,4,Vente,269000.0,1800.0,SAINT-JEAN-DE-NIOST,01,361,Dépendance,0.0,0.0


# Nombre de vente par nature de mutation

In [20]:
con.execute(
                """
                    SELECT
                        nature_mutation,
                        COUNT(nature_mutation) AS nbr_ventes
                    FROM immo_france
                    GROUP BY
                    nature_mutation
                """
            ).df()

,nature_mutation,nbr_ventes
0,Vente,1446737
1,Vente en l'état futur d'achèvement,63908
2,Echange,16133
3,Adjudication,1368
4,Expropriation,161
5,Vente terrain à bâtir,4411


# Calcul des stastitique sur les sous partie de la données

In [25]:
""" 
    Calcul du montant total progressif des valeur_foncière  dans l'Essonne et afficher les dans une colonne 
"""
con.execute(
                """
                    SELECT 
                        nature_mutation, 
                        valeur_fonciere, 
                        code_departement,
                        nom_departement,
                        nom_region,
                        SUM(valeur_fonciere)
                    OVER(ORDER BY nature_mutation) AS mtn_total
                    FROM immo_france
                    LEFT JOIN zone_geographique
                    USING(code_departement)
                    WHERE nom_departement = 'Essonne'
                """
            ).df().head(10)

,nature_mutation,valeur_fonciere,code_departement,nom_departement,nom_region,mtn_total
0,Adjudication,73000.0,91,Essonne,Île-de-France,771000.0
1,Adjudication,73000.0,91,Essonne,Île-de-France,771000.0
2,Adjudication,76000.0,91,Essonne,Île-de-France,771000.0
3,Adjudication,200000.0,91,Essonne,Île-de-France,771000.0
4,Adjudication,200000.0,91,Essonne,Île-de-France,771000.0
5,Adjudication,76000.0,91,Essonne,Île-de-France,771000.0
6,Adjudication,73000.0,91,Essonne,Île-de-France,771000.0
7,Echange,20000.0,91,Essonne,Île-de-France,13248460.5
8,Echange,52687.5,91,Essonne,Île-de-France,13248460.5
9,Echange,10000.0,91,Essonne,Île-de-France,13248460.5


In [40]:
""" 
    Calcul de la moyenne progressive des valeur_foncière  dans Île-de-France et afficher les dans une colonne 
"""
con.execute(
                   """ 
                        SELECT
                            date_mutation,
                            DAYOFWEEK(date_mutation) AS jour_semaine,
                            MONTH(date_mutation) AS numero_mois,
                            nature_mutation, 
                            valeur_fonciere, 
                            code_departement,
                            nom_departement,
                            nom_region,
                            AVG(valeur_fonciere) OVER(ORDER BY date_mutation) AS myn_prog
                        FROM immo_france
                        LEFT JOIN zone_geographique
                        USING(code_departement)            
                        WHERE nom_region = 'Île-de-France'
                        ORDER BY date_mutation    
                    """
            ).df().head(10)


,date_mutation,jour_semaine,numero_mois,nature_mutation,valeur_fonciere,code_departement,nom_departement,nom_region,myn_prog
0,2023-01-02,1,1,Vente,17300.0,75,Paris,Île-de-France,274696.486614
1,2023-01-02,1,1,Vente,895000.0,75,Paris,Île-de-France,274696.486614
2,2023-01-02,1,1,Vente,895000.0,75,Paris,Île-de-France,274696.486614
3,2023-01-02,1,1,Vente,468000.0,75,Paris,Île-de-France,274696.486614
4,2023-01-02,1,1,Vente,468000.0,75,Paris,Île-de-France,274696.486614
5,2023-01-02,1,1,Vente,92500.0,75,Paris,Île-de-France,274696.486614
6,2023-01-02,1,1,Vente,32000.0,75,Paris,Île-de-France,274696.486614
7,2023-01-02,1,1,Vente,1245161.4,75,Paris,Île-de-France,274696.486614
8,2023-01-02,1,1,Vente,1245161.4,75,Paris,Île-de-France,274696.486614
9,2023-01-02,1,1,Vente,276000.0,75,Paris,Île-de-France,274696.486614


In [124]:
""" 
    Calcul de la moyenne progressive des valeur_foncière sur les 7 derniers jours dans l'Essonne et afficher les dans une colonne 
"""

con.execute(
             """ 
                SELECT
                    date_mutation,
                    DAYOFWEEK(date_mutation) AS jour_semaine,
                    MONTH(date_mutation) AS numero_mois,
                    nature_mutation, 
                    valeur_fonciere, 
                    code_departement,
                    nom_departement,
                    nom_region,
                    CAST(AVG(valeur_fonciere) OVER(ORDER BY date_mutation) AS INTEGER) AS myn_prog,
                    CAST(AVG(valeur_fonciere) OVER(
                            ORDER BY date_mutation
                            ROWS BETWEEN 6 PRECEDING and CURRENT ROW
                        ) AS INTEGER) AS myn_prog_7_days
                    FROM immo_france
                    LEFT JOIN zone_geographique
                    USING(code_departement)            
                    WHERE nom_departement = 'Essonne'
                    ORDER BY date_mutation              

             """
            ).df().head(5)

,date_mutation,jour_semaine,numero_mois,nature_mutation,valeur_fonciere,code_departement,nom_departement,nom_region,myn_prog,myn_prog_7_days
0,2023-01-02,1,1,Vente,104000.0,91,Essonne,Île-de-France,179227,104000.0
1,2023-01-02,1,1,Vente,275100.0,91,Essonne,Île-de-France,179227,189550.0
2,2023-01-02,1,1,Vente,275100.0,91,Essonne,Île-de-France,179227,218067.0
3,2023-01-02,1,1,Vente,180000.0,91,Essonne,Île-de-France,179227,208550.0
4,2023-01-02,1,1,Vente,180000.0,91,Essonne,Île-de-France,179227,202840.0


In [125]:
""" 
    Calcul de la moyenne des valeur_foncière en ile de France par département
"""
con.execute(
                """ 
                    SELECT
                        date_mutation,
                        DAYOFWEEK(date_mutation) AS jour_semaine,
                        MONTH(date_mutation) AS numero_mois,
                        nature_mutation, 
                        valeur_fonciere, 
                        code_departement,
                        nom_departement,
                        nom_region,
                        CAST(AVG(valeur_fonciere) OVER(PARTITION BY nom_departement) AS INTEGER) AS mean_dpt
                    FROM immo_france
                    LEFT JOIN zone_geographique
                    USING(code_departement)
                    WHERE nom_region = 'Île-de-France'
                    ORDER BY nom_departement
                                        
                """
           ).df().head(10)

,date_mutation,jour_semaine,numero_mois,nature_mutation,valeur_fonciere,code_departement,nom_departement,nom_region,mean_dpt
0,2023-01-03,2,1,Vente,270000.0,91,Essonne,Île-de-France,809708
1,2023-01-11,3,1,Adjudication,73000.0,91,Essonne,Île-de-France,809708
2,2023-01-04,3,1,Vente,136000.0,91,Essonne,Île-de-France,809708
3,2023-01-04,3,1,Vente,136000.0,91,Essonne,Île-de-France,809708
4,2023-01-03,2,1,Vente,130000.0,91,Essonne,Île-de-France,809708
5,2023-01-02,1,1,Vente,104000.0,91,Essonne,Île-de-France,809708
6,2023-01-04,3,1,Vente,165000.0,91,Essonne,Île-de-France,809708
7,2023-01-04,3,1,Vente,165000.0,91,Essonne,Île-de-France,809708
8,2023-01-04,3,1,Vente,165000.0,91,Essonne,Île-de-France,809708
9,2023-01-04,3,1,Vente,1100000.0,91,Essonne,Île-de-France,809708


In [129]:
""" 
    Calcul de la moyenne (moyenne mobile) des 7 derniers jours des valeur_foncière dans les Yvelines 
"""
con.execute(
                """ 
                    SELECT
                        date_mutation,
                        DAYOFWEEK(date_mutation) AS jour_semaine,
                        MONTH(date_mutation) AS numero_mois,
                        nature_mutation, 
                        valeur_fonciere, 
                        code_departement,
                        nom_departement,
                        nom_region,
                        CAST(AVG(valeur_fonciere) OVER(
                            PARTITION BY jour_semaine
                            ORDER BY date_mutation
                            ROWS BETWEEN 6 PRECEDING 
                            AND CURRENT ROW
                            ) AS INTEGER) AS mean_3last_month
                    FROM immo_france
                    LEFT JOIN zone_geographique
                    USING(code_departement)
                    WHERE (nom_departement = 'Yvelines' AND jour_semaine = 6 AND numero_mois = 1)                         
                """
           ).df().head(10)


,date_mutation,jour_semaine,numero_mois,nature_mutation,valeur_fonciere,code_departement,nom_departement,nom_region,mean_3last_month
0,2023-01-07,6,1,Vente,172500.0,78,Yvelines,Île-de-France,172500
1,2023-01-07,6,1,Vente,172500.0,78,Yvelines,Île-de-France,172500
2,2023-01-07,6,1,Vente,172500.0,78,Yvelines,Île-de-France,172500
3,2023-01-07,6,1,Vente,105000.0,78,Yvelines,Île-de-France,155625
4,2023-01-14,6,1,Vente,224000.0,78,Yvelines,Île-de-France,169300
5,2023-01-14,6,1,Vente,224000.0,78,Yvelines,Île-de-France,178417
6,2023-01-14,6,1,Vente,255000.0,78,Yvelines,Île-de-France,189357
7,2023-01-14,6,1,Vente,255000.0,78,Yvelines,Île-de-France,201143
8,2023-01-14,6,1,Vente,255000.0,78,Yvelines,Île-de-France,212929
9,2023-01-14,6,1,Vente,125000.0,78,Yvelines,Île-de-France,206143
